In [ ]:
## このプログラムは「Digit Recognizer」において
## 作成したノートブックにおいて動作します

# !を付けることでpipコマンドを実行し、Hyperasをインストール。
!pip install hyperas

In [ ]:
# Hyperasはここでインポートする
from hyperopt import hp
from hyperopt import Trials, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
import numpy as np

def prepare_data():
    """データを用意する
    
    """
    # prepare_data()とcreate_model()で使用する
    # 外部ライブラリはここでインポートする
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import KFold
    from tensorflow.keras.utils import to_categorical
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Activation, Dropout

    # train.csvを読み込んでpandasのDataFrameに格納
    train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
    train_x = train.drop(['label'], axis=1) # trainから画像データを抽出
    train_y = train['label']                # trainから正解ラベルを抽出
    test_x = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

    # trainのデータを学習データとテストデータに分ける
    kf = KFold(n_splits=4, shuffle=True, random_state=71)
    tr_idx, va_idx = list(kf.split(train_x))[0]
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # 画像のピクセル値を255.0で割って0～1.0の範囲にしてnumpy.arrayに変換
    tr_x, va_x = np.array(tr_x / 255.0), np.array(va_x / 255.0)
    
    # 正解ラベルをone-hot表現に変換
    tr_y = to_categorical(tr_y, 10)
    va_y = to_categorical(va_y, 10)

    return tr_x, tr_y, va_x, va_y

def create_model(tr_x, tr_y):
    """モデルを生成する
    
    """
    # Sequentialオブジェクトを生成
    model = Sequential()

    # 第1層のユニット数は784
    model.add(Dense(
        784,
        input_dim=tr_x.shape[1],
        activation={{choice(['tanh', 'relu'])}}
    ))
    # 第1層のドロップアウトを0.2～0.4の範囲で探索
    model.add(Dropout(
        {{quniform(0.2, 0.4, 0.05)}}
        ))
    
    # 第2層のユニット数は200。
    model.add(Dense(
        200,
        activation={{choice(['tanh', 'relu'])}}
    ))
    # 第2層のドロップアウトを0.2～0.4の範囲で探索
    model.add(Dropout(
        {{quniform(0.2, 0.4, 0.05)}}
        ))
        
    # 第3層のユニット数は25。
    model.add(Dense(
        25,
        activation={{choice(['tanh', 'relu'])}}
    ))
    # 第3層のドロップアウトを0.2～0.4の範囲で探索。
    model.add(Dropout(
        {{quniform(0.2, 0.4, 0.05)}}
        ))
        
    # 出力層を配置する。活性化関数はSoftmaxで固定
    model.add(Dense(10, activation="softmax"))

    # モデルのコンパイル。
    # オプティマイザーはRMSpropで固定。
    model.compile(loss="categorical_crossentropy",
                  optimizer='rmsprop',
                  metrics=["accuracy"])

    # 学習回数は20回
    epoch = 20
    # ミニバッチのサイズを100と200で試す
    batch_size = {{choice([100, 200])}}
    result = model.fit(tr_x, tr_y,
                       epochs=epoch,
                       batch_size=batch_size,
                       validation_data=(va_x, va_y),
                       verbose=0)

    # 簡易的に訓練時の結果を出力する
    validation_acc = np.amax(result.history['val_accuracy']) 
    print('Best validation acc of epoch:', validation_acc)
    
    # validation_accの値を最小化するように探索する
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

# 探索の実行。試行回数は100とする
best_run, best_model = optim.minimize(model=create_model,
                                      data=prepare_data,
                                      algo=tpe.suggest,
                                      max_evals=100,
                                      eval_space=True,
                                      notebook_name='__notebook_source__',
                                      trials=Trials())

In [ ]:
## 以下、実行するタイミングによって書籍とは異なる結果になります

# 最も精度が優れていたモデルを出力
print(best_model.summary())
# 最も精度が優れていたパラメーター値を出力
print(best_run)

# 探索したモデルでテストデータを検証する
_, _, va_x, va_y = prepare_data()
val_loss, val_acc = best_model.evaluate(va_x, va_y)
print("val_loss: ", val_loss) # 損失を出力
print("val_acc: ", val_acc)   # 精度を出力